In [1]:
def deco(func):
    def inner():
        print('running inner()')
    return inner

@deco
def target():
    print("running targert()")

target()

running inner()


In [2]:
target

<function __main__.deco.<locals>.inner()>

In [5]:
def logger(func):
    def wrapper(*args, **kwargs):
        print(f"[LOG] {func.__name__} called with args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"[LOG] {func.__name__} returned {result}")
        return result
    return wrapper

@logger
def add(a, b):
    return a+b

In [6]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

@register
def f3():
    print('running f3()')

def main():
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()

running register(<function f1 at 0x7dae4a32f9c0>)
running register(<function f2 at 0x7dae4a32f6a0>)
running register(<function f3 at 0x7dae4a32f380>)
running main()
registry -> [<function f1 at 0x7dae4a32f9c0>, <function f2 at 0x7dae4a32f6a0>, <function f3 at 0x7dae4a32f380>]
running f1()
running f2()
running f3()


In [8]:
promos = []

def promotion(promo_func):
    """promotion() decorator는 promo_func를 promos 리스트에 추가 후 그대로 반환"""
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order):
    """충성도 1000점 이상 고객 5% 할인"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item_promo(order):
    """20개 이상 동일 상품 구매시 10% 할인"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order):
    """10종류 이상 상품 구매시 7% 할인"""
    distinct_items = {item.prodect for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    """최대 할인 금액"""
    return max(promo(order) for promo in promos)

In [9]:
def f1(a):
    print(a)
    print(b)

f1(3)

3


NameError: name 'b' is not defined

In [10]:
b = 6
f1(3)

3
6


In [11]:
b = 6
def f2(a):
    print(a)
    print(b)
    b = 9

f2(3)

3


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

In [15]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    b = 9

f3(3)

3
6


9

In [16]:
b

9

In [18]:
f3(3)

3
9


In [19]:
b = 30
f3(3)

3
30


In [20]:
class Averager():
    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)

In [21]:
avg = Averager()
avg(10)

10.0

In [22]:
avg(11)

10.5

In [23]:
avg(12)

11.0

In [24]:
def make_averager():
    series = []

    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total/len(series)

    return averager

In [25]:
avg = make_averager()
avg(10)

10.0

In [26]:
avg(11)

10.5

In [27]:
avg(12)

11.0

In [28]:
avg.__code__.co_varnames

('new_value', 'total')

In [29]:
avg.__code__.co_freevars

('series',)

In [30]:
avg.__closure__

(<cell at 0x7dae490f5ff0: list object at 0x7dae48f9cec0>,)

In [32]:
avg.__closure__[0].cell_contents

[10, 11, 12]

In [33]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

UnboundLocalError: cannot access local variable 'count' where it is not associated with a value

In [34]:
def make_averager():
    count = 0
    total = 0

    def averager(new_value):
        nonlocal count, total
        count += 1
        total += new_value
        return total / count

    return averager

avg = make_averager()
avg(10)

10.0

In [35]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [36]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12317234s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000062s] factorial(1) -> 1
[0.00001308s] factorial(2) -> 2
[0.00001977s] factorial(3) -> 6
[0.00002620s] factorial(4) -> 24
[0.00003261s] factorial(5) -> 120
[0.00004053s] factorial(6) -> 720
6! = 720


In [37]:
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        arg_list = []
        name = func.__name__
        if args:
            arg_list.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = ['%s=%r' % (k, w) for k, w in sorted (kwargs.items())]
            arg_list.append(', '.join(pairs))
        arg_str = ', '.join(arg_list)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [38]:
@clock
def snooze(seconds):
    time.sleep(seconds)

@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

if __name__ == '__main__':
    print('*'*40, 'Calling snooze(.123)')
    snooze(.123)
    print('*'*40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12316704s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000048s] factorial(1) -> 1
[0.00001359s] factorial(2) -> 2
[0.00002003s] factorial(3) -> 6
[0.00002646s] factorial(4) -> 24
[0.00003958s] factorial(5) -> 120
[0.00004768s] factorial(6) -> 720
6! = 720


In [39]:
factorial.__name__

'factorial'